In [ ]:
import pandas as pd
import numpy as np
import os
import os.path
from os.path import join

In [ ]:
df_ref = pd.read_csv(r'T:\ABM\ABM_FY22\RSM\VisionEval\Model\Update_Automations\SrcOnSanVE\geo\mgra13_city_xref.csv')
azone_ref = df_ref[['mgra','City']]
azone_sorter = [12,19,18,7,15,1,14,6,13,17,4,16,5,9,10,3,11,2,8]

In [ ]:
def azone_hhsize_targets (azone_ref):
    azone_ref.columns = ['mgra', 'Geo']
    input_path = r"T:\RTP\2021RP\2021rp_draft_v2\abm_runs"
    years = ['2016', '2035build']
    data = []
    for year in years:
        hh = pd.read_csv(os.path.join(input_path, year, 'input','households.csv'),
                       usecols = ["hhid", #Unique Housheold ID
                                 "mgra", #MGRA of household
                                 "persons", #Number of persons in household
                                 "unittype"]) #household unit type (0 = non-GQ household)
        hh = hh[hh['unittype'] == 0] #non-group quarters households
        hh = hh.merge(azone_ref, on = 'mgra', how = "left")
        df = pd.DataFrame(hh.groupby('Geo')['persons'].mean()).reset_index()
        df['1phh'] =pd.DataFrame(hh[hh['persons'] == 1].groupby('Geo')['hhid'].count()).reset_index()[['hhid']]
        df['allhh'] =pd.DataFrame(hh.groupby('Geo')['hhid'].count()).reset_index()[['hhid']]
        df['Prop1PerHh'] = df['1phh']/df['allhh']
        df.Geo = df.Geo.astype("category")
        df.Geo.cat.set_categories(azone_sorter, inplace=True)
        df = df.sort_values(["Geo"])
        df['Year'] = year[:4]
        data.append(df)
    df_final = pd.concat(data)
    df_final = df_final.rename({'persons':'AveHhSize'}, axis=1)
    df_final = df_final[['Geo','Year','AveHhSize','Prop1PerHh']]
    df_final.to_csv('azone_hhsize_targets.csv', index=False)

In [ ]:
def azone_per_cap_inc (azone_ref):
    azone_ref.columns = ['mgra', 'Geo']
    input_path = r"T:\RTP\2021RP\2021rp_draft_v2\abm_runs"
    years = ['2016', '2035build']
    data = []
    for year in years:
        hh = pd.read_csv(os.path.join(input_path, year, 'input','households.csv'),
                       usecols = ["mgra", #MGRA of household
                                 "hinc", #Household income
                                 "persons", #Number of persons in household
                                 "unittype"]) #household unit type (0 = non-GQ household)
        hh = hh.merge(azone_ref, on = 'mgra', how = "left")
        hh['hinc_perCap'] = hh['hinc']/hh['persons']
        df1 =pd.DataFrame(hh[hh['unittype'] == 0].groupby('Geo')['hinc_perCap'].mean()).reset_index()
        df2 =pd.DataFrame(hh[hh['unittype'] == 1].groupby('Geo')['hinc_perCap'].mean()).reset_index()
        df = df1.merge(df2, on = "Geo", how = "left")
        df.Geo = df.Geo.astype("category")
        df.Geo.cat.set_categories(azone_sorter, inplace=True)
        df = df.sort_values(["Geo"])
        df['Year'] = year[:4]
        data.append(df)
    df_final = pd.concat(data)
    df_final = df_final.rename({'hinc_perCap_x': 'HHIncomePC.2010', 'hinc_perCap_y':'GQIncomePC.2010'}, axis=1)
    df_final['GQIncomePC.2010'] = df_final['GQIncomePC.2010'].fillna(0)
    df_final = df_final[['Geo','Year','HHIncomePC.2010','GQIncomePC.2010']].sort_values(by=['Year','Geo'])
    df_final.to_csv('azone_per_cap_inc.csv', index=False)